In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

openai = OpenAI(
    base_url=f"{endpoint}",
    api_key=api_key
)

python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 8


In [3]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [4]:
def push(messaege: str):
    print("Sending push notification...")
    data = {
        "token": pushover_token,
        "user": pushover_user,
        "message": messaege
    }
    response = requests.post(pushover_url, data=data)
    print(f"Push notification sent with status code {response.status_code}")

In [5]:
def record_user_detail(name: str, email: str, feedback: str):
    # Basic hygiene to avoid empty/whitespace-only inputs
    name = name.strip()
    email = email.strip()
    feedback = feedback.strip()

    if not name or not email or not feedback:
        return {"ok": False, "message": "name, email, and feedback are required"}
    if "@" not in email:
        return {"ok": False, "message": "email appears invalid"}

    push_error = None
    try:
        push(f"New feedback received from {name}({email}): {feedback}")
    except Exception as exc:
        push_error = str(exc)  # Log push failure but continue to persist feedback

    try:
        with open("feedback.txt", "a") as f:
            f.write(f"Name: {name}\nEmail: {email}\nFeedback: {feedback}\n\n")
    except Exception as exc:
        return {
            "ok": False,
            "message": "failed to save feedback",
            "push_error": push_error,
            "error": str(exc),
        }

    return {"ok": True, "message": "Thank you for your feedback!", "push_error": push_error}


In [6]:
def record_unknown_question(question: str):
    question = question.strip()

    if not question:
        return {"ok": False, "message": "question is required"}

    push_error = None
    try:
        push(f"New unknown question received: {question}")
    except Exception as exc:
        push_error = str(exc)  # Log push failure but continue to persist question

    try:
        with open("unknown_questions.txt", "a") as f:
            f.write(f"Question: {question}\n\n")
    except Exception as exc:
        return {
            "ok": False,
            "message": "failed to save question",
            "push_error": push_error,
            "error": str(exc),
        }

    return {"ok": True, "message": "Your question has been recorded.", "push_error": push_error}

In [7]:
""" turn the record_user_detail(name: str, email: str, feedback: str) function into a JSON object 
for the llm to understand the parameters and their types. """
record_user_detail_schema = {
    "name": "record_user_detail",
    "description": "Records user feedback and sends a push notification.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the user providing feedback.",
            },
            "email": {
                "type": "string",
                "description": "The email address of the user providing feedback.",
            },
            "feedback": {
                "type": "string",
                "description": "The feedback provided by the user.",
            },
        },
        "required": ["name", "email", "feedback"],
    },
}


In [8]:
""" turn the record_unknown_question(question: str) function into a JSON object 
for the llm to understand the parameters and their types. """
record_unknown_question_schema = {
    "name": "record_unknown_question",
    "description": "Records an unknown question that you cannot answer and sends a push notification.",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The unknown question to be recorded.",
            },
        },
        "required": ["question"],
    },
}

In [9]:
tools = [{"type": "function", "function": record_unknown_question_schema},
         {"type": "function", "function": record_user_detail_schema}]

In [10]:
tools

[{'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': 'Records an unknown question that you cannot answer and sends a push notification.',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': 'The unknown question to be recorded.'}},
    'required': ['question']}}},
 {'type': 'function',
  'function': {'name': 'record_user_detail',
   'description': 'Records user feedback and sends a push notification.',
   'parameters': {'type': 'object',
    'properties': {'name': {'type': 'string',
      'description': 'The name of the user providing feedback.'},
     'email': {'type': 'string',
      'description': 'The email address of the user providing feedback.'},
     'feedback': {'type': 'string',
      'description': 'The feedback provided by the user.'}},
    'required': ['name', 'email', 'feedback']}}}]

In [11]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [12]:
reader = PdfReader("me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Hassan"

In [13]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [18]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-oss-120b", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [19]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Tool called: record_unknown_question
Sending push notification...
Push notification sent with status code 200
Tool called: record_unknown_question
Sending push notification...
Push notification sent with status code 200
